# Atelier sur la génétique statistique et l'épidémiologie génétique STAGE-Québec
## Thème 2 - Phénotypes moléculaires en épidémiologie génétique

Par Marc-André Legault (Université de Montréal) et Qihuang Zhang (Université McGill)

**31 juillet - 1er août 2025**

### Introduction

Ce carnet présente la deuxième analyse TWAS (Transcriptome-Wide Association Study) utilisant **FUSION**, une approche alternative à la méthode S-PrediXcan démontrée précédemment. En appliquant FUSION aux mêmes statistiques sommaires GWAS, vous gagnerez de l'expérience avec différentes méthodologies TWAS et apprendrez à comparer leurs résultats respectifs.

**Méthodologie FUSION** : FUSION emploie une approche bayésienne pour combiner plusieurs modèles de prédiction et fournit des métriques de qualité complètes pour chaque association gène-trait. Cette analyse complémentaire améliorera la compréhension de comment différentes méthodologies TWAS peuvent donner des insights variés à partir de données d'entrée identiques.

### Comprendre la structure des données FUSION

L'analyse utilise une organisation de données structurée qui facilite les analyses FUSION :

<pre>
/data/FUSION/
├── gwas/      # Statistiques sommaires GWAS (données diabète de l'analyse précédente)
├── ld/        # Panneaux de référence LD (population 1000 Genomes EUR)
└── models/    # Modèles de prédiction d'expression génique pré-entraînés
    ├── GTExv8_EUR_Pancreas/    # Modèles du tissu pancréas de GTEx
    └── YFS_BLOOD_RNAARR/       # Modèles du tissu sanguin de Young Finns Study
</pre>

**Composantes :**
- **Données GWAS** : Statistiques sommaires GWAS diabète analysées précédemment avec S-PrediXcan
- **Référence LD** : Patrons de déséquilibre de liaison spécifiques à la population requis pour l'inférence statistique précise
- **Modèles de prédiction** : Poids pré-entraînés pour estimer l'expression génique à partir des variants génétiques

Chaque répertoire `models/` contient :
- Fichiers `*.wgt.RDat` : Poids de prédiction pour les gènes individuels
- Un fichier `.pos` : Liste maîtresse des gènes disponibles avec coordonnées génomiques

### Exécution de FUSION : Analyse initiale avec les modèles de tissu sanguin

La première analyse emploie des modèles d'expression génique du tissu **sang total** (Young Finns Study). Le tissu sanguin fournit une couverture large car il est facilement accessible et extensivement étudié.

**Note computationnelle** : L'analyse traite tous les 22 autosomes, avec une déclaration de rupture après le chromosome 2 incluse pour l'efficacité temporelle. Les résultats génomiques complets sont disponibles pour examen, et l'analyse complète peut être exécutée en retirant la déclaration de rupture.

In [ ]:
%%bash
for chrom in $(seq 1 22); do
    Rscript /workshop/software/FUSION/FUSION.assoc_test.R \
        --sumstats /workshop/data/FUSION/gwas/Mahajan.NatGen2022.DIAMANTE-EUR.ldsc_ss.FUSION_ref.tsv \
        --weights /workshop/data/FUSION/models/YFS_BLOOD_RNAARR/YFS.BLOOD.RNAARR.pos \
        --weights_dir /workshop/data/FUSION/models/YFS_BLOOD_RNAARR/ \
        --ref_ld_chr /workshop/data/FUSION/ld/1000G.EUR. \
        --chr $chrom \
        --out results/fusion_YFS_$chrom.tsv
        
    if (( $chrom > 2 )); then
        # Commentez la ligne ci-dessous pour exécuter tous les chromosomes,
        # sinon seulement les deux premiers seront utilisés.
        break
    fi
done

### Comparaison de tissus : Analyse du pancréas GTEx

L'analyse subséquente emploie des modèles de prédiction du tissu **pancréas** utilisant les données GTEx. Ceci fournit une opportunité de comparaison directe parce que l'analyse utilise les mêmes données d'expression du pancréas GTEx qui supportaient l'analyse S-PrediXcan.

In [ ]:
%%bash
for chrom in $(seq 1 22); do
    Rscript /workshop/software/FUSION/FUSION.assoc_test.R \
        --sumstats /workshop/data/FUSION/gwas/Mahajan.NatGen2022.DIAMANTE-EUR.ldsc_ss.FUSION_ref.tsv \
        --weights /workshop/data/FUSION/models/GTExv8_EUR_Pancreas/GTExv8.EUR.Pancreas.pos \
        --weights_dir /workshop/data/FUSION/models/GTExv8_EUR_Pancreas \
        --ref_ld_chr /workshop/data/FUSION/ld/1000G.EUR. \
        --chr $chrom \
        --out results/fusion_GTEx_pancreas_$chrom.tsv
        
    if (( $chrom > 2 )); then
        # Commentez la ligne ci-dessous pour exécuter tous les chromosomes,
        # sinon seulement les deux premiers seront utilisés.
        break
    fi
done

### Organisation des résultats pour l'analyse

Pour une analyse complète, les résultats spécifiques aux chromosomes devraient être combinés en fichiers uniques. Le code suivant démontre comment fusionner les sorties spécifiques aux chromosomes, bien qu'il soit commenté pour cette session d'atelier qui se concentre sur un sous-ensemble de chromosomes.

In [ ]:
%%bash
# Fusionner les fichiers (GTEx Pancréas).
# cp results/fusion_GTEx_pancreas_1.tsv results/fusion_GTEx_pancreas.tsv
# for chrom in $(seq 2 22); do
#     cat results/fusion_GTEx_pancreas_${chrom}.tsv | sed 1d >> results/fusion_GTEx_pancreas.tsv
# done
#
# Fusionner les fichiers (YFS).
# cp results/fusion_YFS_1.tsv results/fusion_YFS.tsv
# for chrom in $(seq 2 22); do
#     cat results/fusion_YFS_${chrom}.tsv | sed 1d >> results/fusion_YFS.tsv
# done

### Structure de sortie FUSION : Interprétation des résultats

L'analyse FUSION génère des résultats TWAS en format de valeurs séparées par tabulations (TSV). Comprendre la structure de sortie est essentiel pour l'analyse en aval et l'interprétation. La commande suivante examine les en-têtes de colonnes :

In [ ]:
%%bash
head -n 1 /workshop/local/results/fusion_GTEx_pancreas_1.tsv | tr '\t' '\n' | nl

### Interprétation de la sortie FUSION : Métriques clés

Comprendre les colonnes de sortie de FUSION est essentiel pour une interprétation appropriée des résultats. Les métriques les plus importantes incluent :

**Résultats TWAS primaires :**
- **TWAS.Z, TWAS.P** : Statistiques d'association gène-trait centrales - scores Z et valeurs P
- **ID** : Identifiant de gène (le format varie selon le jeu de données - peut être des symboles de gènes ou des ID Ensembl)

**Métriques d'évaluation de qualité :**
- **MODELCV.R2** : R² de validation croisée indiquant à quel point les variants génétiques prédisent l'expression génique (valeurs plus élevées indiquent une plus grande fiabilité)
- **MODELCV.PV** : Valeur P pour la performance du modèle de prédiction
- **NSNP, NWGT** : Nombre de SNP utilisés et nombre avec des poids non-zéro

**Contexte additionnel :**
- **BEST.GWAS.Z** : Signal GWAS individuel le plus fort dans la région du gène (pour comparaison avec le signal TWAS)
- **EQTL.R2** : Capacité prédictive du top eQTL pour l'expression
- **HSQ** : Estimation d'héritabilité pour l'expression génique

Exemple de résultats démontrant ces métriques :

In [ ]:
%%bash
awk '{print $1, $4, $5, $9, $10, $11, $19, $20}' /workshop/local/results/fusion_GTEx_pancreas_1.tsv | head

In [ ]:
%%bash
wc -l ./results/fusion_GTEx_pancreas.tsv
wc -l ./results/fusion_YFS.tsv

<div class="alert alert-info">
<h4>Exercice d'analyse 1 : Évaluation de la signification pangénomique</h4>

<strong>Objectif :</strong> Identifier les gènes atteignant la signification pangénomique en utilisant la correction de Bonferroni dans les deux types de tissus.

<strong>Tâches :</strong>
1. Calculer le seuil de signification : 0,05 divisé par le nombre de gènes testés par tissu
2. Quantifier les gènes répondant à ce critère strict dans chaque tissu
3. Évaluer les implications pour les effets génétiques spécifiques aux tissus versus partagés

<strong>Méthode :</strong> Utiliser les comptes de lignes montrés ci-dessus pour déterminer le nombre total de gènes testés, puis appliquer le filtrage approprié.

<!--
Solution:
awk 'NR == 1 || $20 <= 0.05/5765 { print $3, $19, $20 }' results/fusion_GTEx_pancreas.tsv
awk 'NR == 1 || $20 <= 0.05/4628 { print $3, $19, $20 }' results/fusion_YFS.tsv

# 119 significatifs pour GTEx pancréas
# 85 significatifs pour YFS
//-->

</div>

<div class="alert alert-info">
<h4>Exercice d'analyse 2 : Évaluation de la qualité du modèle</h4>

<strong>Objectif :</strong> Évaluer comment MODELCV.R2 devrait influencer la confiance dans les résultats TWAS.

<strong>Considérations :</strong>
- Interprétation des valeurs R² faibles (par ex., 0,01) concernant la fiabilité du signal TWAS
- Évaluation des valeurs TWAS.P hautement significatives quand MODELCV.R2 est faible
- Établissement de seuils R² raisonnables pour le filtrage des résultats
- Facteurs biologiques qui peuvent contribuer à une précision de prédiction faible pour certains gènes
</div>

<div class="alert alert-info">
<h4>Exercice d'analyse 3 : Patrons d'association spécifiques aux tissus</h4>

**Objectif :** Évaluer les patrons différentiels d'association génique entre les tissus sanguins et pancréatiques et leurs implications pour la biologie du diabète.

**Domaines d'investigation :**
1. **Associations partagées** : Identifier les gènes avec des associations significatives dans les deux tissus et évaluer la cohérence de direction d'effet
2. **Associations spécifiques aux tissus** : Déterminer les gènes significatifs dans seulement un tissu et évaluer les facteurs explicatifs potentiels
3. **Comparaison d'ampleur d'effet** : Pour les gènes significatifs dans les deux tissus, comparer les forces d'association
4. **Interprétation biologique** : Évaluer si les patrons spécifiques aux tissus s'alignent avec la pathophysiologie établie du diabète

**Approches analytiques :**
- Générer des graphiques de dispersion comparant les scores TWAS.Z entre les tissus
- Évaluer les différences systématiques dans la qualité du modèle de prédiction (MODELCV.R2) entre les tissus

</div>

### Prochaines étapes

Dans le [prochain carnet](3a-compare_TWAS_models.ipynb), nous comparerons les résultats de TWAS et S-PrediXcan et explorerons leur sensibilité et spécificité.